<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentatorResults/blob/main/rawDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TL;DR
This notebook is part 1 of a two-part series that reproduces our results from the TotalSegmentator Segmentations on NLST. We've made available over 125k CT scans with approximately 9.5 Million segmentations. For each segmentation, we extracted 28 shape and first-order radiomics features using pyradiomics. We encoded these segmentations and features into DICOM SEG objects and DICOM Structured Reports respectively. We also saved the general module features from pyradiomics in JSON format. All data is publicly available and can be run without logging into any cloud. The notebook is self contained and can be run not only on colab but any jupyterlab/notebook run time.




## Introduction


We recently made available TotalSegmentator[1] Segmentations on NLST for over 125k CT scans with approximately 9.5 Million segmentations[2] on Imaging Data Commons[3]. In addition, for each segmentation, we extracted 28 shape and first-order radiomics features using pyradiomics[4]. We encoded these Segmentations into DICOM SEG objects and shape and first-order features into DICOM Structured Reports. Moreover, we also saved pyradiomics general module features in JSON format.

To help get a sense of the quality of these segmentations, we proposed four rule-based heuristics without the use of AI/ML. This notebook is part 1 of a two-part series that helps reproduce our results. All data is made publicly available and can be run without the requirement of logging into any cloud.



## Data Used


This notebook generates or makes available the links of the following data we used:

- **PerframeFunctionalGroupsSequence**: The DICOM SEG object contains the DICOM attribute PerframeFunctionalGroupsSequence which encodes segment and its corresponding slice locations. Due to BigQuery's limit of 1 MB per value in a cell, DICOM Segmentation Objects generated from TotalSegmentator/dcmqi that contained the DICOM attribute PerFrameFunctionalGroupsSequence over 1 MB were dropped from the BigQuery metadata table. To extract this attribute, we developed a workflow on [Terra](https://dockstore.org/my-workflows/github.com/ImagingDataCommons/CloudSegmentator/perFrameFunctionalGroupSequenceExtractionOnTerra). We unnested this attribute and are making it available to the public as a parquet file.

- **jsonRadiomics**: While extracting the 28 radiomics features mentioned in the paper, we also saved the raw output from pyradiomics into a JSON file. Pyradiomics provides `general features` along with any first or shape features. We extracted and pooled the radiomics features in JSON files and provided them as a parquet file. To generate this file, we used a Terra workflow, pushed the combined table to BigQuery, and then exported it as parquet files, consolidating it into one parquet file.

- **bodyPartAndLaterality**: This is an intermediate table which contains information about the body part segmented by TotalSegmentator, segment number, source CT series, and its Laterality.

- **segmentation_completeness_check**: This checks whether a segment contains at least one slice below and above the segmentation.

- **Minimum Volume from Voxel Summation**: This checks whether a segment has at least 5 mL volume.

- **laterality_check**: This checks if laterality (left vs right) is correctly assigned by TotalSegmentator.

- **qualitative_checks_and_quant_measurements**: This is the result of combining all the heuristics along with 28 radiomics features for each segment along with 2 general module features VoxelNum and Connected Component Volumes.


## References



1. Wasserthal, J., Breit, H., Meyer, M.T., Pradella, M., Hinck, D., Sauter, A.W., Heye, T., Boll, D., Cyriac, J., Yang, S., Bach, M., & Segeroth, M. (2022). TotalSegmentator: Robust Segmentation of 104 Anatomic Structures in CT Images. Radiology. Artificial intelligence, 5 5, e230024.
2. Thiriveedhi, V. K., Krishnaswamy, D., Clunie, D., Pieper, S., Kikinis, R., & Fedorov, A. (2024). Cloud-based large-scale curation of medical imaging data using AI segmentation. Research square, rs.3.rs-4351526. https://doi.org/10.21203/rs.3.rs-4351526/v1
3. Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S. D., Gibbs, D. L., Bridge, C., Herrmann, M. D., Homeyer, A., Lewis, R., Aerts, H. J. W. L., Krishnaswamy, D., Thiriveedhi, V. K., Ciausu, C., Schacherer, D. P., Bontempi, D., Pihl, T., Wagner, U., Farahani, K., Kim, E., … Kikinis, R. (2023). National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence. Radiographics : a review publication of the Radiological Society of North America, Inc, 43(12), e230180. https://doi.org/10.1148/rg.230180
4. van Griethuysen, J. J. M., Fedorov, A., Parmar, C., Hosny, A., Aucoin, N., Narayan, V., Beets-Tan, R. G. H., Fillion-Robin, J. C., Pieper, S., & Aerts, H. J. W. L. (2017). Computational Radiomics System to Decode the Radiographic Phenotype. Cancer research, 77(21), e104–e107. https://doi.org/10.1158/0008-5472.CAN-17-0339

## Compute Environment details

Initially we ran this notebook on a 64 vCPU, 256 GB RAM [JetStream2](https://jetstream-cloud.org/) instance using local colab runtime as described [here](https://research.google.com/colaboratory/local-runtimes.html). While some steps can be run on a free colab instance (2 vCPUs, 13 GB RAM), some steps that require scanning and holding the entire data in memory are RAM intensive.

Please see (https://docs.jetstream-cloud.org/alloc/overview/) on how researchers in the US can request an ACCESS allocation for free.

Here's a link to list of free jupyter runtime options:

https://www.dataschool.io/cloud-services-for-jupyter-notebook/

## Base Tables

Download the bases time in parquet file format containing DICOM attribute `PerFrameFunctionalGroupsSequence` attribute and `jsonRadiomics` table

In [1]:
!wget  https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/nlst_totalseg_perframe.parquet
!wget  https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/json_radiomics.parquet

## Install duckdb
duckdb is an in-memomy process database that can work with highly complex data with very small footprint. Link to their GitHub repo: https://github.com/duckdb/duckdb

In [5]:
%%capture
!pip install --upgrade duckdb s5cmd pandas pyarrow

We will be querying agains the IDC tables stored in AWS buckets in parquet files directly. Even though these tables are public, we must create a dummy secret to enable querying via duckdb

In [1]:
import duckdb
gcs_public_sql='''
CREATE SECRET (
    TYPE S3,
    KEY_ID '',
    SECRET ''
)'''
duckdb.query(gcs_public_sql)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

## Generate Segmentation Completeness check table.
Let's do this in three steps:
1. Extract extract the bodyPart and laterality from dicom_all table
2. Get minimum and maximum z component in a series
3. Get minimum and maximum z compoent for a bodyPart

Combine the three subsets to determine segmentation completeness based on presence of at least one slice below and above the bodyPart




![Alt text](https://raw.githubusercontent.com/vkt1414/CloudSegmentatorResults/main/src/images/segmentation_completeness_check.png
)

Depending on the compute available, this step may take anywhere from 5 mins on a (64 vCPUs) or ~2 hrs (8 vCPUs) on colab pro or ~8-10 hrs (estimated) on colab free version (2 vCPUs)





In [24]:
import subprocess
import pandas as pd
import duckdb
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def process_file(file_path):
    sql = f'''
    SELECT
    StudyInstanceUID,
    rss.SeriesInstanceUID rssSIUD,
    ss.segmentDescription,
    REGEXP_EXTRACT(ss.segmentDescription,': (.*)' , 1) totalsegSegmentLabel,
    ss.SegmentNumber segment_number,
    ss_sptcs.CodeMeaning bodyPart,
    ss_sptcs_sptmcs.CodeMeaning laterality
    FROM read_parquet('{file_path}')
    LEFT JOIN UNNEST(ReferencedSeriesSequence) t1(rss) ON TRUE
    LEFT JOIN UNNEST(SegmentSequence) t2(ss) ON TRUE
    LEFT JOIN UNNEST(ss.SegmentedPropertyTypeCodeSequence) t3 (ss_sptcs) ON TRUE
    LEFT JOIN UNNEST(ss_sptcs.SegmentedPropertyTypeModifierCodeSequence) t4(ss_sptcs_sptmcs) ON TRUE
    WHERE
    collection_id  IN ('nlst')
    and MODALITY IN ('SEG')
    and analysis_result_id IN ('TotalSegmentator-CT-Segmentations')
    '''
    return duckdb.query(sql).df()

# Define the GCS bucket directory
bucket_dir = 's3://idc-open-metadata/bigquery_export/idc_v18/dicom_all/'

# Run the s5cmd command and capture the output
proc = subprocess.run(
    ['s5cmd','--no-sign-request', '--endpoint-url', 'https://s3.amazonaws.com', 'ls', bucket_dir],
    capture_output=True, text=True
)

# Extract paths of each parquet file
files = [bucket_dir + line.split()[3] for line in proc.stdout.splitlines() if line]

# Use a ProcessPoolExecutor to process files in parallel
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_file, files), total=len(files)))

# Concatenate all the results into a single DataFrame
bodyPartAndLaterality = pd.concat(results)
bodyPartAndLaterality


100%|██████████| 5001/5001 [1:50:44<00:00,  1.33s/it]


,StudyInstanceUID,rssSIUD,segmentDescription,totalsegSegmentLabel,segment_number,bodyPart,laterality
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.283584872380...,1.3.6.1.4.1.14519.5.2.1.7009.9004.594299167694...,TotalSegmentator 15 : lung_upper_lobe_right,lung_upper_lobe_right,14,Upper lobe of lung,Right
1,1.2.840.113654.2.55.12530111708225589708623715...,1.2.840.113654.2.55.23696367762517311761791651...,TotalSegmentator 14 : lung_lower_lobe_left,lung_lower_lobe_left,14,Lower lobe of lung,Left
2,1.2.840.113654.2.55.12530111708225589708623715...,1.2.840.113654.2.55.23696367762517311761791651...,TotalSegmentator 70 : rib_right_1,rib_right_1,53,First rib,Right
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.797190834968...,1.3.6.1.4.1.14519.5.2.1.7009.9004.126198274629...,TotalSegmentator 79 : rib_right_10,rib_right_10,65,Tenth rib,Right
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.275725030103...,1.3.6.1.4.1.14519.5.2.1.7009.9004.254705695638...,TotalSegmentator 64 : rib_left_7,rib_left_7,49,Seventh rib,Left
...,...,...,...,...,...,...,...
2168,1.2.840.113654.2.55.24743520996724964481035604...,1.2.840.113654.2.55.75891416193884470011646121...,TotalSegmentator 30 : vertebrae_T5,vertebrae_T5,25,T5 vertebra,None
2169,1.2.840.113654.2.55.24743520996724964481035604...,1.2.840.113654.2.55.75891416193884470011646121...,TotalSegmentator 45 : heart_atrium_left,heart_atrium_left,34,Left atrium,None
2170,1.2.840.113654.2.55.26096434357414097924788200...,1.2.840.113654.2.55.26122393554409295038882991...,TotalSegmentator 34 : vertebrae_T1,vertebrae_T1,30,T1 vertebra,None
2171,1.2.840.113654.2.55.18798690957573916889401685...,1.2.840.113654.2.55.19300254091202538238567307...,TotalSegmentator 28 : vertebrae_T7,vertebrae_T7,23,T7 vertebra,None


This subset is useful in the next check as well and so we will save it as a parquet file, which we also made it available on GitHub

In [25]:
bodyPartAndLaterality.to_parquet('bodyPartAndLaterality.parquet', compression='zstd')

If you would like to skip the previous step, you can download the file we made available on GitHub

In [ ]:
#!wget https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/bodyPartAndLaterality.parquet

This step retrieves the z positions of the terminal slices of a segmentation

In [26]:
body_parts_min_max_z_values_sql='''
  SELECT
    ReferencedSeriesSequence_SeriesInstanceUID rssSIUD,
    SegmentIdentificationSequence_ReferencedSegmentNumber segment_number,
    MAX(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) segmentation_max_z_position,
    MIN(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) segmentation_min_z_position
  FROM
    'nlst_totalseg_perframe.parquet' pffgs
  GROUP BY
    rssSIUD,
    segment_number
'''
body_parts_min_max_z_values=duckdb.sql(body_parts_min_max_z_values_sql).df()
body_parts_min_max_z_values

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,rssSIUD,segment_number,segmentation_max_z_position,segmentation_min_z_position
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.111187839890...,32,1853.000000,1847.596924
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.111187839890...,56,1853.000000,1795.367188
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.111187839890...,68,1853.000000,1831.387695
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.111187839890...,16,1710.718994,1615.264648
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.111187839890...,17,1753.943604,1573.841064
...,...,...,...,...
9565700,1.3.6.1.4.1.14519.5.2.1.7009.9004.209911511744...,16,-174.250000,-271.750000
9565701,1.3.6.1.4.1.14519.5.2.1.7009.9004.209911511744...,23,-224.250000,-276.750000
9565702,1.3.6.1.4.1.14519.5.2.1.7009.9004.209911511744...,32,-44.250000,-66.750000
9565703,1.3.6.1.4.1.14519.5.2.1.7009.9004.209911511744...,33,-44.250000,-49.250000


This step retrieves the z-positions of the terminal slices of a series or scan as a whole

In [27]:
series_min_max_z_values_sql='''
  SELECT
    ReferencedSeriesSequence_SeriesInstanceUID rssSIUD,
    MAX(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) series_max_z_position,
    MIN(CAST(PlanePositionSequence_ImagePositionPatient[3] AS FLOAT)) series_min_z_position
  FROM
    'nlst_totalseg_perframe.parquet' pffgs
  GROUP BY
    rssSIUD
'''
series_min_max_z_values=duckdb.sql(series_min_max_z_values_sql).df()
series_min_max_z_values

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,rssSIUD,series_max_z_position,series_min_z_position
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.224287822985...,-17.000,-343.250
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.223209007849...,75.500,-234.500
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.222965016822...,-28.500,-344.500
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.223476572084...,-11.500,-330.500
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.265569431878...,1819.000,1505.000
...,...,...,...
126048,1.3.6.1.4.1.14519.5.2.1.7009.9004.210551425060...,34.125,-329.875
126049,1.3.6.1.4.1.14519.5.2.1.7009.9004.208879238832...,486.000,123.000
126050,1.3.6.1.4.1.14519.5.2.1.7009.9004.209843747414...,370.000,13.000
126051,1.3.6.1.4.1.14519.5.2.1.7009.9004.211494825952...,38.250,-299.750


Finally, we put the previous three tables together to determine if there were any slices below and above a segmentation, to evaluate segmentation completeness check

In [28]:
segmentation_completeness_check_sql='''
SELECT
  DISTINCT series_min_max_z_values.rssSIUD,
  segmentDescription,
  bodyPartAndLaterality.totalsegSegmentLabel,
  bodyPartAndLaterality.segment_number,
  bodyPartAndLaterality.bodyPart,
  bodyPartAndLaterality.laterality,
  series_min_max_z_values.series_max_z_position,
  series_min_max_z_values.series_min_z_position,
  body_parts_min_max_z_values.segmentation_max_z_position,
  body_parts_min_max_z_values.segmentation_min_z_position,
  CASE
    WHEN ((ABS(series_max_z_position-segmentation_max_z_position)>0 AND ABS(series_min_z_position-segmentation_min_z_position)>0)) THEN 'may_be_segmented_fully'
  ELSE
  'may_not_be_segmented_fully'
END
  AS segmentation_completeness_check,
FROM
  'bodyPartAndLaterality.parquet' bodyPartAndLaterality
JOIN
  series_min_max_z_values
ON
  bodyPartAndLaterality.rssSIUD=series_min_max_z_values.rssSIUD
JOIN
  body_parts_min_max_z_values
ON
  bodyPartAndLaterality.rssSIUD=body_parts_min_max_z_values.rssSIUD
  AND bodyPartAndLaterality.segment_number= body_parts_min_max_z_values.segment_number
'''
segmentation_completeness_check_df=duckdb.sql(segmentation_completeness_check_sql).df()
segmentation_completeness_check_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,rssSIUD,segmentDescription,totalsegSegmentLabel,segment_number,bodyPart,laterality,series_max_z_position,series_min_z_position,segmentation_max_z_position,segmentation_min_z_position,segmentation_completeness_check
0,1.2.840.113654.2.55.16664558776859946313482997...,TotalSegmentator 45 : heart_atrium_left,heart_atrium_left,36,Left atrium,None,1321.500000,1005.500000,1181.500000,1121.500000,may_be_segmented_fully
1,1.2.840.113654.2.55.16712317052575288700299884...,TotalSegmentator 5 : liver,liver,5,Liver,None,-876.500000,-1202.500000,-1118.500000,-1202.500000,may_not_be_segmented_fully
2,1.2.840.113654.2.55.16638773163266253675111253...,TotalSegmentator 64 : rib_left_7,rib_left_7,50,Seventh rib,Left,1572.099976,1260.099976,1460.099976,1358.099976,may_be_segmented_fully
3,1.2.840.113654.2.55.16638773163266253675111253...,TotalSegmentator 75 : rib_right_6,rib_right_6,61,Sixth rib,Right,1572.099976,1260.099976,1492.099976,1402.099976,may_be_segmented_fully
4,1.2.840.113654.2.55.16638773163266253675111253...,TotalSegmentator 32 : vertebrae_T3,vertebrae_T3,29,T3 vertebra,None,1572.099976,1260.099976,1550.099976,1506.099976,may_be_segmented_fully
...,...,...,...,...,...,...,...,...,...,...,...
9565549,1.2.840.113654.2.55.12298145805444245180037308...,TotalSegmentator 12 : adrenal_gland_left,adrenal_gland_left,12,Adrenal gland,Left,1755.500000,1428.500000,1482.500000,1449.500000,may_be_segmented_fully
9565550,1.2.840.113654.2.55.12266053890814099623657014...,TotalSegmentator 70 : rib_right_1,rib_right_1,55,First rib,Right,1766.000000,1448.000000,1766.000000,1712.000000,may_not_be_segmented_fully
9565551,1.2.840.113654.2.55.10977101001506277912314714...,TotalSegmentator 72 : rib_right_3,rib_right_3,57,Third rib,Right,-392.500000,-686.500000,-430.500000,-504.500000,may_be_segmented_fully
9565552,1.2.840.113654.2.55.13947313737928626336084974...,TotalSegmentator 23 : vertebrae_T12,vertebrae_T12,18,T12 vertebra,None,-100.000000,-388.000000,-336.000000,-384.000000,may_be_segmented_fully


In [29]:
segmentation_completeness_check_df.to_parquet('segmentation_completeness_table.parquet', compression='zstd')

In [ ]:
#!wget https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/bodyPartAndLaterality.parquet

## Check if Laterality was correctly assigned by TotalSegmentator
We checked by comparing the x coordinate of pyradiomics general module feature Center of Mass. Because we did not save general module features into a DICOM Structured Reports, we made them available as a parquet file instead.

![Alt text](https://raw.githubusercontent.com/vkt1414/CloudSegmentatorResults/main/src/images/laterarlity%20check.jpg)


In [32]:
laterality_check_sql='''
with temp as
(SELECT
  CT_seriesInstanceUID,
  organ,
  REGEXP_REPLACE(REGEXP_REPLACE(organ, 'left', ''), 'right','') as organ_with_out_laterality,
  diagnostics_Mask_original_CenterOfMass[1] AS x_coordinate,
  CASE
  WHEN organ like '%left%' THEN 'left'
  wHEN organ like '%right%' THEN 'right'
  else null
  end as laterality
FROM
  'json_radiomics.parquet'
),
temp2 as (
select
CT_seriesInstanceUID,
organ organ_left,
x_coordinate x_coordinate_left,
lead(organ) over (partition by CT_SeriesInstanceUID, organ_with_out_laterality order by organ) organ_right,
lead(x_coordinate) over (partition by CT_SeriesInstanceUID, organ_with_out_laterality order by organ) x_coordinate_right
from temp where laterality is not null
order by 1,2
)
select distinct
temp2.*,
CASE WHEN x_coordinate_left > x_coordinate_right THEN 'pass'
     ELSE 'fail'
END AS laterality_check,
from temp2
where organ_right is not null

'''
laterality_check=duckdb.sql(laterality_check_sql).df()
laterality_check

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CT_SeriesInstanceUID,organ_left,x_coordinate_left,organ_right,x_coordinate_right,laterality_check
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.418628086802...,rib_left_11,69.636055,rib_right_11,-83.431357,pass
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.418628086802...,rib_left_6,100.519947,rib_right_6,-89.695422,pass
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.418679164104...,iliopsoas_left,13.654773,iliopsoas_right,-38.180337,pass
3,1.3.6.1.4.1.14519.5.2.1.7009.9004.418679164104...,rib_left_11,79.502772,rib_right_11,-96.072242,pass
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.418679164104...,rib_left_12,53.235175,rib_right_12,-67.817863,pass
...,...,...,...,...,...,...
2822843,1.3.6.1.4.1.14519.5.2.1.7009.9004.418390132209...,heart_atrium_left,19.208025,heart_atrium_right,-16.008044,pass
2822844,1.3.6.1.4.1.14519.5.2.1.7009.9004.418390132209...,rib_left_10,128.955014,rib_right_10,-99.257532,pass
2822845,1.3.6.1.4.1.14519.5.2.1.7009.9004.418390132209...,rib_left_3,100.501065,rib_right_3,-84.067201,pass
2822846,1.3.6.1.4.1.14519.5.2.1.7009.9004.418390132209...,scapula_left,140.372243,scapula_right,-121.776464,pass


Lets write the results to a parquet file

In [33]:
laterality_check.to_parquet('laterality_check_table.parquet', compression='zstd')


## Qualitative Checks Table

In this table, we combined the results of all three checks above, and pulled pyradiomics general features voxel num and connected volumes attributes. The fourth heuristic 'Minimum Volume from Voxel Summation' will be added later on, when we join the quantitative features

![Alt text](https://raw.githubusercontent.com/vkt1414/CloudSegmentatorResults/main/src/images/qual-checks.jpg
)

We will use idc-index python package to retrieve some series level info. Learn more about idc-index at https://github.com/ImagingDataCommons/idc-index

In [34]:
%%capture
!pip install idc-index

In [35]:
from idc_index import index
client=index.IDCClient()

Lets rename index in idc_index as idc_index_df so we can use it for queries using duckdb

In [36]:
idc_index_df=client.index

In [38]:
import duckdb
qual_checks_table='''
WITH temp as(
SELECT
  DISTINCT
  rssSIUD AS CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  idc.seriesNumber,
  bodyPart,
  sct.laterality,
  laterality_check,
  CASE
    WHEN sct.segmentation_completeness_check ='may_be_segmented_fully' THEN 'pass'
    ELSE 'fail'
  END AS segmentation_completeness,
  jr.diagnostics_Mask_original_VolumeNum as connected_volumes,
  diagnostics_Mask_original_VoxelNum as voxel_num
FROM
  'segmentation_completeness_table.parquet' sct
LEFT JOIN
  'json_radiomics.parquet' jr
ON
  sct.rssSIUD=jr.CT_seriesInstanceUID
AND sct.totalsegSegmentLabel= jr.organ

JOIN idc_index_df idc
ON idc.SeriesInstanceUID = sct.rssSIUD

LEFT JOIN 'laterality_check_table.parquet' lct
  ON sct.rssSIUD = lct.CT_SeriesInstanceUID
  and sct.totalsegSegmentLabel = lct.organ_left
),
temp2 as(
SELECT *  FROM temp
UNION BY NAME
SELECT distinct
  temp.CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  seriesNumber,
  bodyPart,
  laterality,
  segmentation_completeness,
  connected_volumes,
  voxel_num,
  lct.laterality_check
FROM
temp
LEFT JOIN 'laterality_check_table.parquet' lct
  ON temp.CT_SeriesInstanceUID = lct.CT_SeriesInstanceUID
  and temp.totalsegSegmentLabel = lct.organ_right
)
SELECT
  CT_SeriesInstanceUID,
  totalsegSegmentLabel,
  StudyInstanceUID,
  seriesNumber,
  bodyPart,
  laterality,
  segmentation_completeness,
  connected_volumes,
  voxel_num,
  max(laterality_check) laterality_check
FROM
  temp2
group by 1,2,3,4,5,6,7,8,9
'''
qual_checks_table=duckdb.sql(qual_checks_table).df()
qual_checks_table


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CT_SeriesInstanceUID,totalsegSegmentLabel,StudyInstanceUID,SeriesNumber,bodyPart,laterality,segmentation_completeness,connected_volumes,voxel_num,laterality_check
0,1.2.840.113654.2.55.28621153544054705437823203...,rib_left_1,1.2.840.113654.2.55.47613407579656102240134667...,6608,First rib,Left,pass,1.0,8441.0,pass
1,1.2.840.113654.2.55.27601635490337947013203602...,rib_right_7,1.2.840.113654.2.55.25024335655177524281267801...,5,Seventh rib,Right,pass,5.0,12803.0,pass
2,1.2.840.113654.2.55.27597401707226338700613265...,rib_right_2,1.2.840.113654.2.55.93031386845264789335609664...,2,Second rib,Right,pass,1.0,23997.0,pass
3,1.2.840.113654.2.55.21393561452791621778907295...,vertebrae_T12,1.2.840.113654.2.55.33148653181105855779285606...,3,T12 vertebra,None,pass,1.0,46054.0,None
4,1.2.840.113654.2.55.33895310199277463601154569...,vertebrae_T3,1.2.840.113654.2.55.83020794784870462814821574...,4,T3 vertebra,None,pass,1.0,10750.0,None
...,...,...,...,...,...,...,...,...,...,...
9565549,1.2.840.113654.2.55.16620675898916185195521091...,iliopsoas_left,1.2.840.113654.2.55.15270894507535219542702110...,5,Iliopsoas muscle,Left,fail,1.0,124.0,pass
9565550,1.3.6.1.4.1.14519.5.2.1.7009.9004.272588167616...,rib_left_7,1.3.6.1.4.1.14519.5.2.1.7009.9004.151922662590...,3,Seventh rib,Left,pass,1.0,34771.0,pass
9565551,1.3.6.1.4.1.14519.5.2.1.7009.9004.380235045354...,pancreas,1.3.6.1.4.1.14519.5.2.1.7009.9004.186292625351...,2,Pancreas,None,fail,1.0,53200.0,None
9565552,1.3.6.1.4.1.14519.5.2.1.7009.9004.487175390891...,pulmonary_artery,1.3.6.1.4.1.14519.5.2.1.7009.9004.254805603371...,2,Pulmonary artery,None,pass,1.0,120501.0,None


Lets write the output to a parquet file

In [39]:
qual_checks_table.to_parquet('qual_checks_table.parquet', compression='zstd')


## Flatenned quantitative measurements table

Because we are pivoting the table we use aggregate functions, and aggregate queries must go through all rows before returning any results. This resulted in extremely high RAM usage.

In [2]:
def flatten_radiomics_feature(radiomics_feature):
    sql=f'''
    with temp as(
    SELECT
      PatientID,
      segmentationInstanceUID,
      sourceSegmentedSeriesUID,
      findingSite.CodeMeaning bodyPart,
      lateralityModifier.CodeMeaning Laterality,
      --MAX(
        CASE WHEN Quantity.CodeMeaning = '{radiomics_feature}' THEN Value END AS "{radiomics_feature}"
    FROM
      's3://idc-open-metadata/bigquery_export/idc_v18/quantitative_measurements/*.parquet'
    WHERE
      SeriesDescription LIKE '%TotalSeg%'
       )
    SELECT
      PatientID,
      segmentationInstanceUID,
      sourceSegmentedSeriesUID,
      bodyPart,
      Laterality,
      MAX("{radiomics_feature}") as "{radiomics_feature}"
    FROM
      temp
    GROUP BY
    1,2,3,4,5
    '''
    flattened_quantitative_measurements_1=duckdb.sql(sql).to_parquet(f'flattened_quantitative_measurements_{radiomics_feature}.parquet', compression='zstd')



In [3]:
radiomics_feature_list=['10th percentile',
                        '90th percentile',
                        'Elongation',
                        'Energy',
                        'Flatness',
                        'Intensity Histogram Entropy',
                        'Intensity histogram uniformity',
                        'Interquartile range',
                        'Kurtosis',
                        'Least Axis in 3D Length',
                        'Major Axis in 3D Length',
                        'Maximum 3D Diameter of a Mesh',
                        'Maximum grey level',
                        'Mean',
                        'Mean absolute deviation',
                        'Median',
                        'Minimum grey level',
                        'Minor Axis in 3D Length',
                        'Range',
                        'Robust mean absolute deviation',
                        'Root mean square',
                        'Skewness',
                        'Sphericity',
                        'Surface Area of Mesh',
                        'Surface to Volume Ratio',
                        'Variance',
                        'Volume from Voxel Summation',
                        'Volume of Mesh'
]


In [4]:
import duckdb
from tqdm  import tqdm
for radiomics_feature in tqdm(radiomics_feature_list):
  flatten_radiomics_feature(radiomics_feature)


  0%|          | 0/28 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  4%|▎         | 1/28 [06:39<2:59:48, 399.57s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  7%|▋         | 2/28 [18:07<4:06:37, 569.13s/it]

...

 93%|█████████▎| 26/28 [4:23:25<27:18, 819.04s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 96%|█████████▋| 27/28 [4:37:09<13:40, 820.63s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

100%|██████████| 28/28 [4:49:01<00:00, 619.35s/it]


In [1]:
import polars as pl
import pandas as pd

# List of all your parquet files
files = [
    'flattened_quantitative_measurements_10th percentile.parquet',
    'flattened_quantitative_measurements_90th percentile.parquet',
     'flattened_quantitative_measurements_Elongation.parquet',
    'flattened_quantitative_measurements_Energy.parquet',
    'flattened_quantitative_measurements_Flatness.parquet',
    'flattened_quantitative_measurements_Intensity Histogram Entropy.parquet',
    'flattened_quantitative_measurements_Intensity histogram uniformity.parquet',
    'flattened_quantitative_measurements_Interquartile range.parquet',
    'flattened_quantitative_measurements_Kurtosis.parquet',
    'flattened_quantitative_measurements_Least Axis in 3D Length.parquet',
    'flattened_quantitative_measurements_Major Axis in 3D Length.parquet',
    'flattened_quantitative_measurements_Maximum 3D Diameter of a Mesh.parquet',
    'flattened_quantitative_measurements_Maximum grey level.parquet',
    'flattened_quantitative_measurements_Mean.parquet',
    'flattened_quantitative_measurements_Mean absolute deviation.parquet',
    'flattened_quantitative_measurements_Median.parquet',
    'flattened_quantitative_measurements_Minimum grey level.parquet',
    'flattened_quantitative_measurements_Minor Axis in 3D Length.parquet',
    'flattened_quantitative_measurements_Range.parquet',
    'flattened_quantitative_measurements_Robust mean absolute deviation.parquet',
    'flattened_quantitative_measurements_Root mean square.parquet',
    'flattened_quantitative_measurements_Skewness.parquet',
    'flattened_quantitative_measurements_Sphericity.parquet',
    'flattened_quantitative_measurements_Surface Area of Mesh.parquet',
    'flattened_quantitative_measurements_Surface to Volume Ratio.parquet',
    'flattened_quantitative_measurements_Variance.parquet',
    'flattened_quantitative_measurements_Volume from Voxel Summation.parquet',
    'flattened_quantitative_measurements_Volume of Mesh.parquet',
]

import polars as pl

# Read the first file
df = pl.read_parquet(files[0])

# Define the common columns for joining
common_columns = ["PatientID", "segmentationInstanceUID", "sourceSegmentedSeriesUID", "bodyPart"]

# Loop over the rest of the files and join
for file in files[1:]:
    df_temp = pl.read_parquet(file)

    # Split the dataframes into two based on the nullity of 'Laterality'
    df_null = df.filter(pl.col('Laterality').is_null())
    df_null = df_null.drop('Laterality')
    df_not_null = df.filter(pl.col('Laterality').is_not_null())
    df_temp_null = df_temp.filter(pl.col('Laterality').is_null())
    df_temp_null = df_temp_null.drop('Laterality')
    df_temp_not_null = df_temp.filter(pl.col('Laterality').is_not_null())

    # Perform the join operation separately on the split dataframes
    df_null = df_null.join(df_temp_null, on=common_columns)
    df_not_null = df_not_null.join(df_temp_not_null, on=common_columns + ['Laterality'])


    # Now you can safely concatenate the dataframes
    df = pl.concat([df_null, df_not_null], how='diagonal')

flattened_quantitative_measurements = df
flattened_quantitative_measurements


PatientID,segmentationInstanceUID,sourceSegmentedSeriesUID,bodyPart,10th percentile,90th percentile,Elongation,Energy,Flatness,Intensity Histogram Entropy,Intensity histogram uniformity,Interquartile range,Kurtosis,Least Axis in 3D Length,Major Axis in 3D Length,Maximum 3D Diameter of a Mesh,Maximum grey level,Mean,Mean absolute deviation,Median,Minimum grey level,Minor Axis in 3D Length,Range,Robust mean absolute deviation,Root mean square,Skewness,Sphericity,Surface Area of Mesh,Surface to Volume Ratio,Variance,Volume from Voxel Summation,Volume of Mesh,Laterality
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""112808""","""1.2.276.0.7230…","""1.2.840.113654…","""Left atrium""",-74.0,154.0,0.864,5.59186994e8,0.603,3.88,0.08,118.0,0.101,33.508,55.553,63.402,429.0,40.61,70.596,41.0,-394.0,48.018,823.0,49.374,97.699,-0.014,0.722,10609.064,0.168,7895.861,63075.059,62994.668,null
"""102098""","""1.2.276.0.7230…","""1.2.840.113654…","""Left atrium""",-19.0,76.0,0.686,1.95749476e8,0.535,2.677,0.189,48.0,0.844,34.667,64.825,75.186,265.0,28.98,29.831,31.0,-165.0,44.5,430.0,20.312,48.207,-0.338,0.71,11565.693,0.165,1484.076,69983.353,69913.043,null
"""205274""","""1.2.276.0.7230…","""1.3.6.1.4.1.14…","""Left ventricle…",17.0,81.0,0.578,8.3728641e8,0.51,2.13,0.268,34.0,0.17,40.416,79.251,94.52,181.0,48.844,20.196,49.0,-93.0,45.841,274.0,14.056,55.092,0.033,0.66,15510.226,0.159,649.461,97525.064,97473.448,null
"""205253""","""1.2.276.0.7230…","""1.3.6.1.4.1.14…","""Myocardium""",-278.0,70.0,0.864,1.3928e10,0.779,3.484,0.16,65.0,6.549,71.368,91.597,98.665,261.0,-49.368,141.022,32.0,-1024.0,79.116,1285.0,38.967,236.376,-2.755,0.31,38323.208,0.315,53436.434,121718.262,121686.646,null
"""124741""","""1.2.276.0.7230…","""1.2.840.113654…","""Left atrium""",-2.0,81.0,0.739,2.15651324e8,0.562,2.513,0.212,43.0,41.24,35.253,62.709,70.547,1123.0,39.683,26.565,41.0,-278.0,46.357,1401.0,18.043,53.353,1.274,0.717,11580.014,0.163,1271.863,71208.659,71134.442,null
"""205575""","""1.2.276.0.7230…","""1.3.6.1.4.1.14…","""Left ventricle…",20.0,75.0,0.642,3.94628478e8,0.576,1.911,0.311,28.0,0.13,45.332,78.71,88.95,144.0,48.014,17.111,48.0,-76.0,50.524,220.0,12.066,52.628,-0.044,0.714,16456.449,0.137,464.319,119846.436,119767.999,null
"""217091""","""1.2.276.0.7230…","""1.3.6.1.4.1.14…","""Left atrium""",11.0,64.0,0.768,1.65899828e8,0.522,1.881,0.323,27.0,0.564,30.078,57.622,64.13,127.0,37.905,16.598,38.0,-89.0,44.242,216.0,11.552,43.428,-0.231,0.73,9498.837,0.175,449.219,54360.695,54314.218,null
"""121240""","""1.2.276.0.7230…","""1.2.840.113654…","""Pulmonary arte…",-9.0,86.0,0.881,3.39425276e8,0.387,2.72,0.188,49.0,53.63,37.176,96.024,121.786,192.0,37.723,31.495,41.0,-983.0,84.571,1175.0,20.379,60.959,-4.276,0.486,17523.03,0.237,2292.973,74062.116,73917.08,null
"""218381""","""1.2.276.0.7230…","""1.3.6.1.4.1.14…","""Left atrium""",-7.0,78.0,0.877,2.42233447e8,0.569,2.522,0.208,44.0,21.433,32.688,57.461,67.813,186.0,34.795,27.121,36.0,-601.0,50.37,787.0,18.526,50.92,-2.015,0.736,10786.156,0.162,1382.159,66509.859,66455.991,null


Combine all four parts into one

In [2]:
flattened_quantitative_measurements.write_parquet('flattened_quantitative_measurements.parquet', compression='zstd')


## Qualitative checks and quantitative measurements
Finally, lets put qualitative checks and quantitative measurements together.

![Alt text](https://raw.githubusercontent.com/vkt1414/CloudSegmentatorResults/main/src/images/qual-quant-checks.jpg
)

In [ ]:
# !wget https://github.com/ImagingDataCommons/CloudSegmentatorResults/releases/download/0.0.1/qual_checks_table.parquet

In [6]:
import duckdb
sql='''
SELECT
PatientID,
--qct.*exclude (CT_SeriesInstanceUID),
qct.*,
qmt.*exclude(PatientID,segmentationInstanceUID,sourceSegmentedSeriesUID, bodyPart, Laterality),
CASE WHEN "Volume from Voxel Summation" > 5000 THEN 'pass'
     ELSE 'fail'
END AS volume_from_voxel_summation_check
from

'qual_checks_table.parquet' qct

join

'flattened_quantitative_measurements.parquet' qmt

on qct.CT_seriesInstanceUID=qmt.sourceSegmentedSeriesUID
and qct.bodyPart=qmt.bodyPart
and ((qct.laterality = qmt.laterality) OR (qct.laterality IS NULL AND qmt.laterality IS NULL))
'''
duckdb.sql(sql).to_parquet('qual_checks_and_quantitative_measurements.parquet', compression='zstd')

#duckdb.sql(sql).df().columns

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

parquet_files=[

    'bodyPartAndLaterality.parquet',
    'flat_quantitative_measurements.parquet',
    'json_radiomics.parquet',
    'laterality_check_table.parquet',
    'nlst_totalseg_perframe.parquet',
    'qual_checks_and_quantitative_measurements.parquet',
    'qual_checks_table.parquet',
    'segmentation_completeness_table.parquet'
]
for parquet_file in parquet_files:
    print(parquet_file)
    table = pq.read_table(parquet_file)

    # Get the schema
    schema = table.schema

    # Create a DataFrame from the schema
    schema_df = pd.DataFrame({
        'Column Name': [field.name for field in schema],
        'Data Type': [field.type for field in schema]
    })

    # Write the schema DataFrame to a CSV file
    schema_df.to_csv(f'{parquet_file[:-8]}_schema.csv', index=False)


